In [4]:
import tkinter as tk
from tkinter import messagebox
import sqlite3

# Database setup
conn = sqlite3.connect('quiz_app.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT UNIQUE,
                    password TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS results (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT,
                    category TEXT,
                    score INTEGER)''')
conn.commit()

quizzes = {
    "DSA": [
        {"question": "What is the time complexity of binary search?", 
         "options": ["O(n)", "O(log n)", "O(n^2)", "O(1)"], 
         "answer": "O(log n)"},
        {"question": "Which data structure is LIFO?", 
         "options": ["Queue", "Stack", "Array", "Heap"], 
         "answer": "Stack"},
        {"question": "What does a queue follow?", 
         "options": ["FIFO", "LIFO", "FILO", "None"], 
         "answer": "FIFO"},
        {"question": "Which sorting algorithm has the best average-case time complexity?", 
         "options": ["Bubble Sort", "Merge Sort", "Selection Sort", "Insertion Sort"], 
         "answer": "Merge Sort"},
        {"question": "What is the height of a balanced binary tree with n nodes?", 
         "options": ["O(n)", "O(log n)", "O(1)", "O(n^2)"], 
         "answer": "O(log n)"}
    ],
    "DBMS": [
        {"question": "What does SQL stand for?", 
         "options": ["Structured Query Language", "Simple Query Language", "Secure Query Language", "Sequential Query Language"], 
         "answer": "Structured Query Language"},
        {"question": "What is a primary key?", 
         "options": ["Unique identifier for a row", "Column index", "Foreign key", "Secondary key"], 
         "answer": "Unique identifier for a row"},
        {"question": "Which of the following is a NoSQL database?", 
         "options": ["MySQL", "MongoDB", "PostgreSQL", "Oracle"], 
         "answer": "MongoDB"},
        {"question": "What does ACID stand for in DBMS?", 
         "options": ["Atomicity, Consistency, Isolation, Durability", "Access, Control, Integration, Durability", "Atomicity, Coordination, Isolation, Data"], 
         "answer": "Atomicity, Consistency, Isolation, Durability"},
        {"question": "Which type of join returns all rows from the left table and matching rows from the right table?", 
         "options": ["Inner Join", "Outer Join", "Left Join", "Cross Join"], 
         "answer": "Left Join"}
    ],
    "Python": [
        {"question": "What is the output of 'print(2**3)'?", 
         "options": ["5", "8", "6", "Error"], 
         "answer": "8"},
        {"question": "Which keyword is used to define a function in Python?", 
         "options": ["func", "def", "function", "lambda"], 
         "answer": "def"},
        {"question": "What is the output of 'print(type([]))'?", 
         "options": ["<class 'list'>", "<class 'tuple'>", "<class 'dict'>", "<class 'set'>"], 
         "answer": "<class 'list'>"},
        {"question": "What is the correct way to create a set in Python?", 
         "options": ["{}", "[]", "set()", "set[]"], 
         "answer": "set()"},
        {"question": "Which of the following is used to handle exceptions in Python?", 
         "options": ["if-else", "for-while", "try-except", "None"], 
         "answer": "try-except"}
    ]
}


logged_in_user = None

# Helper function for centering windows
def center_window(window, width, height):
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    window.geometry(f"{width}x{height}+{x}+{y}")

# Helper Functions
def register():
    username = reg_username.get()
    password = reg_password.get()
    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
        messagebox.showinfo("Success", "Registration successful!")
        registration_window.destroy()
    except sqlite3.IntegrityError:
        messagebox.showerror("Error", "Username already exists!")

def login():
    global logged_in_user
    username = login_username.get()
    password = login_password.get()
    cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
    if cursor.fetchone():
        logged_in_user = username
        messagebox.showinfo("Success", "Login successful!")
        login_window.destroy()
        show_main_menu()
    else:
        messagebox.showerror("Error", "Invalid username or password.")

def attempt_quiz(category):
    global question_index, score
    question_index = 0
    score = 0
    current_quiz = quizzes[category]
    
    def next_question():
        global question_index, score
        if selected_option.get() == current_quiz[question_index]['answer']:
            score += 1
        
        question_index += 1
        if question_index < len(current_quiz):
            display_question(question_index)
        else:
            messagebox.showinfo("Quiz Finished", f"You scored {score}/{len(current_quiz)}")
            cursor.execute("INSERT INTO results (username, category, score) VALUES (?, ?, ?)", (logged_in_user, category, score))
            conn.commit()
            quiz_window.destroy()
            show_main_menu()

    def display_question(index):
        question_label.config(text=current_quiz[index]['question'])
        for i, option in enumerate(current_quiz[index]['options']):
            options_buttons[i].config(text=option, value=option)

    quiz_window = tk.Toplevel(bg="#f5f5f5")
    quiz_window.title(f"{category} Quiz")
    center_window(quiz_window, 600, 400)
    question_label = tk.Label(quiz_window, text="", font=("Arial", 16, "bold"), bg="#f5f5f5", fg="#333")
    question_label.pack(pady=20)
    selected_option = tk.StringVar()
    options_buttons = [tk.Radiobutton(quiz_window, text="", variable=selected_option, font=("Arial", 14), bg="#f5f5f5", fg="#444") for _ in range(4)]
    for btn in options_buttons:
        btn.pack(anchor="w", padx=50)
    next_button = tk.Button(quiz_window, text="Next", font=("Arial", 14), bg="#007BFF", fg="white", command=next_question)
    next_button.pack(pady=20)

    display_question(question_index)

def show_result():
    result_window = tk.Toplevel(bg="#f5f5f5")
    result_window.title("Your Results")
    center_window(result_window, 400, 300)
    cursor.execute("SELECT * FROM results WHERE username = ?", (logged_in_user,))
    results = cursor.fetchall()
    if results:
        for result in results:
            tk.Label(result_window, text=f"Category: {result[2]} | Score: {result[3]}", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack(pady=5)
    else:
        tk.Label(result_window, text="No results found.", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack()

def show_main_menu():
    main_menu = tk.Toplevel(bg="#f5f5f5")
    main_menu.title("Quiz Application - Main Menu")
    center_window(main_menu, 400, 400)
    tk.Label(main_menu, text=f"Welcome, {logged_in_user}", font=("Arial", 18, "bold"), bg="#f5f5f5", fg="#007BFF").pack(pady=20)
    tk.Button(main_menu, text="Attempt Quiz - DSA", font=("Arial", 14), bg="#007BFF", fg="white", command=lambda: attempt_quiz("DSA")).pack(pady=10)
    tk.Button(main_menu, text="Attempt Quiz - DBMS", font=("Arial", 14), bg="#007BFF", fg="white", command=lambda: attempt_quiz("DBMS")).pack(pady=10)
    tk.Button(main_menu, text="Attempt Quiz - Python", font=("Arial", 14), bg="#007BFF", fg="white", command=lambda: attempt_quiz("Python")).pack(pady=10)
    tk.Button(main_menu, text="Show Results", font=("Arial", 14), bg="#007BFF", fg="white", command=show_result).pack(pady=10)
    tk.Button(main_menu, text="Exit", font=("Arial", 14), bg="#FF0000", fg="white", command=main_menu.destroy).pack(pady=20)

# Registration Window
def open_registration():
    global reg_username, reg_password, registration_window
    registration_window = tk.Toplevel(bg="#f5f5f5")
    registration_window.title("Register")
    center_window(registration_window, 400, 300)
    tk.Label(registration_window, text="Username:", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack(pady=10)
    reg_username = tk.Entry(registration_window, font=("Arial", 14))
    reg_username.pack()
    tk.Label(registration_window, text="Password:", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack(pady=10)
    reg_password = tk.Entry(registration_window, font=("Arial", 14), show="*")
    reg_password.pack()
    tk.Button(registration_window, text="Register", font=("Arial", 14), bg="#007BFF", fg="white", command=register).pack(pady=20)

# Login Window
def open_login():
    global login_username, login_password, login_window
    login_window = tk.Toplevel(bg="#f5f5f5")
    login_window.title("Login")
    center_window(login_window, 400, 300)
    tk.Label(login_window, text="Username:", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack(pady=10)
    login_username = tk.Entry(login_window, font=("Arial", 14))
    login_username.pack()
    tk.Label(login_window, text="Password:", font=("Arial", 14), bg="#f5f5f5", fg="#333").pack(pady=10)
    login_password = tk.Entry(login_window, font=("Arial", 14), show="*")
    login_password.pack()
    tk.Button(login_window, text="Login", font=("Arial", 14), bg="#007BFF", fg="white", command=login).pack(pady=20)

# Main Window
root = tk.Tk()
root.title("Quiz Application")
center_window(root, 500, 400)
root.configure(bg="#f5f5f5")

tk.Label(root, text="Quiz Application", font=("Arial", 24, "bold"), bg="#f5f5f5", fg="#007BFF").pack(pady=20)
tk.Button(root, text="Register", font=("Arial", 16), bg="#007BFF", fg="white", command=open_registration).pack(pady=10)
tk.Button(root, text="Login", font=("Arial", 16), bg="#007BFF", fg="white", command=open_login).pack(pady=10)
tk.Button(root, text="Exit", font=("Arial", 16), bg="#FF0000", fg="white", command=root.quit).pack(pady=10)

root.mainloop()
